In [1]:
import numpy as np
import pandas as pd
import pyecharts as echarts

In [6]:
dataset1 = pd.read_table('./data/train_20171215.txt')

In [220]:
weekday_list = dataset1[['date','day_of_week']].drop_duplicates().reset_index()
weekday_list['index']=''
weekday_list['is_rest_before']=0
index = 0
weekday = 3
i=0
is_rest = 0
while 1:
    if i<1032:
        if weekday >7:
            weekday =1
        if weekday_list.loc[i,'day_of_week'] == weekday:
            weekday_list.loc[i,'index'] = index
            if (i>0) & (is_rest==1):
                weekday_list.loc[i,'is_rest_before'] = 1
                is_rest = 0
            index +=1
            i+=1
            weekday+=1
        else:
            row={'index':index,'date':'','day_of_week':weekday,'is_rest_before':is_rest}
            weekday_list= weekday_list.append(row,ignore_index=True)
            index += 1
            weekday+=1
            is_rest = 1
    else:
        break     
weekday_list = weekday_list.sort_values(by=['index'])#.reset_index()[['date','day_of_week']]
daily_cnt = dataset1.groupby(['date','day_of_week'])['cnt'].sum().reset_index()
train_set = pd.merge(weekday_list,daily_trade,how='left',on=['date','day_of_week'])
train_set['index'] = train_set['index'].apply(lambda x:x+1)
train_set['cycle_index'] = [int(i/30.5)%12 for i in train_set['index']]
train_set = train_set[train_set.date!='']

In [228]:
train_set.to_csv('./1111.csv')

In [227]:
from sklearn.ensemble import GradientBoostingRegressor as GBR
gbr = GBR()
gbr.fit(train_set[['day_of_week','is_rest_before','MA7','MA30','cycle_index']],train_set.cnt)
# train_set[['day_of_week','is_rest_before','MA7','MA30','cycle_index']]
gbr.predict([6,0,4521,4571,10])

d:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([-414.10408553])

In [205]:
enc = OneHotEncoder()
train_set = enc.fit_transform(train_set)
train_set.

<1001x4190 sparse matrix of type '<class 'numpy.float64'>'
	with 8008 stored elements in Compressed Sparse Row format>

In [162]:
bar = echarts.Bar()
bar.add('',train_set['index'],train_set.cnt,is_datazoom_show=True)
bar

In [4]:
weekday_cnt = dataset1.groupby('day_of_week')['cnt'].sum().reset_index()
bar = echarts.Bar()
bar.add('',weekday_cnt.day_of_week,weekday_cnt.cnt)
bar

In [94]:
daily_cnt = dataset1.groupby('date').agg('sum').cnt
# bar = echarts.Bar()
# bar.add('',daily_cnt.index,daily_cnt.values,is_datazoom_show=True)
# bar

In [95]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    dftest = adfuller(timeseries, autolag='AIC')
    return dftest[1]
    #此函数返回的是p值
test_stationarity(daily_cnt)

3.3890388360843764e-05

In [96]:
MA30 = []
count = 0
sum = 0
for i in range(len(daily_cnt.index)):
    if count<30:
        sum+=daily_cnt.iloc[i]
        count +=1
    else:
        MA30.append(float(sum/30))
        sum = 0
        sum += daily_cnt.iloc[i]
        count = 1    
bar = echarts.Bar()
bar.add('',list(range(len(MA30))),MA30)
bar

In [102]:
grid = echarts.Grid()
overlap = echarts.Overlap()
MA7 = pd.rolling_mean(daily_cnt,window=7) 
MA30 = pd.rolling_mean(daily_cnt,window=30)       
# bar = echarts.Bar()
# bar.add('',list(range(len(daily_cnt))),daily_cnt.values,is_datazoom_show=True)
# overlap.add(bar)
line = echarts.Line()
line.add('daily',list(range(len(daily_cnt))),daily_cnt.values)
line.add('MA7',list(range(len(MA7))),MA7,is_datazoom_show=True)
line.add('MA30',list(range(len(MA30))),MA30,is_datazoom_show=True)
overlap.add(line)
overlap

d:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=7,center=False).mean()
  app.launch_new_instance()
d:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=30,center=False).mean()


date
1               NaN
2               NaN
3               NaN
4               NaN
5               NaN
6               NaN
7       3074.714286
8       3507.285714
9       3889.714286
10      3438.857143
11      2768.714286
12      2681.428571
13      2537.142857
14      2216.428571
15      2641.571429
16      2642.000000
17      2328.714286
18      2292.571429
19      2459.428571
20      2504.285714
21      2618.428571
22      2047.142857
23      1902.285714
24      1904.428571
25      2383.714286
26      2257.000000
27      2240.714286
28      2091.000000
29      2184.285714
30      1981.000000
           ...     
1003    1876.000000
1004    1978.571429
1005    1770.428571
1006    2004.571429
1007    1978.714286
1008    1945.142857
1009    1873.142857
1010    1895.142857
1011    1659.571429
1012    1946.857143
1013    1964.428571
1014    1908.714286
1015    1791.142857
1016    1777.857143
1017    1587.857143
1018    1854.428571
1019    1782.571429
1020    1742.857143
1021    1690.28

In [118]:
daily_trade = dataset1.groupby('date').agg('sum')[['cnt']].reset_index()
daily_trade['MA7'] = MA7
daily_trade['MA30'] = MA30
# daily_trade['cycle_id'] = [int(i/30)%11 for i in range(len(daily_cnt))]
daily_trade = pd.merge(daily_trade,dataset1[['date','day_of_week']],how='left',on='date').drop_duplicates().fillna(0)
pd.merge(traindaily_trade,


,date,cnt,MA7,MA30,day_of_week
0,False,False,False,False,False
2,False,False,False,False,False
4,False,False,False,False,False
9,False,False,False,False,False
14,False,False,False,False,False
19,False,False,False,False,False
24,False,False,False,False,False
29,False,False,False,False,False
34,False,False,False,False,False
39,False,False,False,False,False


In [ ]:
import sklearn as sk
gbr = sk.ensemble.GradientBoostingRegressor()
